In [22]:
import xarray as xrs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cfgrib
import math
import os
import FWI

ds = xrs.open_dataset("2015_abril_maio.grib", engine='cfgrib')
print(ds)

<xarray.Dataset> Size: 629kB
Dimensions:    (time: 61, latitude: 23, longitude: 16)
Coordinates:
  * time       (time) datetime64[ns] 488B 2015-04-01T12:00:00 ... 2015-05-31T...
    surface    float64 8B ...
  * latitude   (latitude) float64 184B 42.2 41.95 41.7 41.45 ... 37.2 36.95 36.7
  * longitude  (longitude) float64 128B 350.2 350.4 350.7 ... 353.4 353.7 353.9
Data variables:
    fbupinx    (time, latitude, longitude) float32 90kB ...
    drtcode    (time, latitude, longitude) float32 90kB ...
    dufmcode   (time, latitude, longitude) float32 90kB ...
    ffmcode    (time, latitude, longitude) float32 90kB ...
    fdsrte     (time, latitude, longitude) float32 90kB ...
    fwinx      (time, latitude, longitude) float32 90kB ...
    infsinx    (time, latitude, longitude) float32 90kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Con

In [21]:
def DistanceTwoPoints(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def find_nearest(lat, lon, latitudes, longitudes):
    min_distance = None
    nearest_point = None

    for lat2, lon2 in zip(latitudes, longitudes):
        distance = DistanceTwoPoints(lat, lon, lat2, lon2)
        if min_distance is None or distance < min_distance:
            min_distance = distance
            nearest_point = (lat2, lon2)

    return min_distance, nearest_point

def find_nearest_index(lat, lon, latitudes, longitudes):
    distances = np.array([DistanceTwoPoints(lat, lon, lat2, lon2) for lat2, lon2 in zip(latitudes, longitudes)])
    sorted_distances_indices = np.argsort(distances)
    nearest_points = [(latitudes[i], longitudes[i]) for i in sorted_distances_indices[:4]]
    min_distances = distances[sorted_distances_indices[:4]]
    return min_distances, nearest_points

def convert_longitude360(lon):
    if lon < 0:
        return lon + 360
    return lon

def convert_longitude(lon):
    if lon > 180:
        return lon - 360
    return lon


def total_precipitation(initial_date, final_date, dmet):
    dmet['hourly.time'] = pd.to_datetime(dmet['hourly.time'])
    dmet_copy = dmet.copy()
    dmet_copy.set_index('hourly.time', inplace=True)
    selected_rows = dmet_copy.loc[initial_date:final_date]
    total_precipitation = selected_rows['hourly.precipitation'].sum()

    return total_precipitation

In [5]:
fwinx_at_time = ds['fwinx'].sel(time='2015-04-29T12:00:00')

print(fwinx_at_time)

latitudes = fwinx_at_time.latitude
longitudes = fwinx_at_time.longitude

# Print the latitudes
print(latitudes.values)
print(len(latitudes.values))
print(longitudes.values)
print(len(longitudes.values))

novas_lat = []
novas_lon = []
for lat in range(len(latitudes)):
    for lon in range(len(longitudes)):
        novas_lat.append(latitudes[lat].values)
        novas_lon.append(longitudes[lon].values)
        



<xarray.DataArray 'fwinx' (latitude: 23, longitude: 16)> Size: 1kB
[368 values with dtype=float32]
Coordinates:
    time       datetime64[ns] 8B 2015-04-29T12:00:00
    surface    float64 8B ...
  * latitude   (latitude) float64 184B 42.2 41.95 41.7 41.45 ... 37.2 36.95 36.7
  * longitude  (longitude) float64 128B 350.2 350.4 350.7 ... 353.4 353.7 353.9
Attributes: (12/28)
    GRIB_paramId:                             260540
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      368
    GRIB_typeOfLevel:                         surface
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    ...                                       ...
    GRIB_name:                                Forest fire weather index
    GRIB_shortName:                           fwinx
    GRIB_units:                               Numeric
    long_name:                                Forest fire weather index
    uni

In [28]:
estacoes = [f for f in os.listdir("estacoes") if f.endswith('.csv')]

for est in estacoes:
    df = pd.read_csv("estacoes/" + est)
    LATITUDE_SING = df.iloc[0]['latitude']
    LONGITUDE_SING = df.iloc[0]['longitude']
    min_distance, nearest_point = find_nearest(LATITUDE_SING, convert_longitude360(LONGITUDE_SING), novas_lat, novas_lon)
    print(min_distance, nearest_point)

    print("Latitude: ", nearest_point[0])
    print("Longitude: ", nearest_point[1])

    fwinx_value = ds['fwinx'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    drtcode_value = ds['drtcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    dufmcode_value = ds['dufmcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    ffmcode_value = ds['ffmcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])

    dmet_filtered = df[df['hourly.time'].str.contains('T12:00')]
    dmet_filtered.to_csv(f'estacoes12/MD_{est}', index=False)
    
    start_index = 0 + 1
    LAT = nearest_point[0]
    FFMCPrev = ffmcode_value.values.item()
    DMCPrev = dufmcode_value.values.item()
    DCPrev = drtcode_value.values.item()
    mts = {
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8:31, 
        9:30, 
        10:31}


    fwix_values = [None] * len(dmet_filtered)
    ffmc_values = [None] * len(dmet_filtered)
    dmc_values = [None] * len(dmet_filtered)
    dc_values = [None] * len(dmet_filtered)
    isi_values = [None] * len(dmet_filtered)
    bui_values = [None] * len(dmet_filtered)


    for i in range(start_index, len(dmet_filtered)):
        MONTH = (dmet_filtered.iloc[i]['hourly.time']).split('-')[1]
        DAY = (dmet_filtered.iloc[i]['hourly.time']).split('-')[2].split('T')[0]

        print(MONTH, DAY)

        TEMP = dmet_filtered.iloc[i]['hourly.temperature_2m']
        RH = dmet_filtered.iloc[i]['hourly.relative_humidity_2m']
        WIND = dmet_filtered.iloc[i]['hourly.wind_speed_10m']


        if(int(DAY) == 1):
            LAST_DAY = mts[int(MONTH) - 1]
            LAST_MONTH = str(int(MONTH) - 1)
            
            if(int(LAST_MONTH) < 10):
                LAST_MONTH = '0' + str(int(LAST_MONTH))
                
            initial_date = f'2015-{LAST_MONTH}-{LAST_DAY}T12:00'
            final_date = f'2015-{MONTH}-{DAY}T12:00'

        elif((int(DAY) < 10) and (int(DAY) > 1)):
            id = '0' + str(int(DAY)-1)
            fd = '0' + str(int(DAY))

            initial_date = f'2015-{MONTH}-{id}T12:00'
            final_date = f'2022-{MONTH}-{fd}T12:00'
        else:
            id = str(int(DAY)-1)
            fd = str(int(DAY))

            initial_date = f'2015-{MONTH}-{id}T12:00'
            final_date = f'2015-{MONTH}-{fd}T12:00'

    
        RAIN = total_precipitation(initial_date, final_date, df)

        FFMCPrev = FWI.FFMC(TEMP,RH,WIND,RAIN,FFMCPrev)
        DMCPrev = FWI.DMC(TEMP,RH,RAIN,DMCPrev,LAT,int(MONTH))
        DCPrev = FWI.DC(TEMP,RAIN,DCPrev,LAT,int(MONTH))
        isi = FWI.ISI(WIND, FFMCPrev)
        bui = FWI.BUI(DMCPrev,DCPrev)
        fwix = FWI.FWI(isi, bui)

        fwix_values[i] = fwix
        ffmc_values[i] = FFMCPrev
        dmc_values[i] = DMCPrev
        dc_values[i] = DCPrev
        isi_values[i] = isi
        bui_values[i] = bui


    dmet_filtered['fwix'] = fwix_values
    dmet_filtered['ffmc'] = ffmc_values
    dmet_filtered['dmc'] = dmc_values
    dmet_filtered['dc'] = dc_values
    dmet_filtered['isi'] = isi_values
    dmet_filtered['bui'] = bui_values


    dmet_filtered.to_csv(f'estacoes12X/fwi_{est}', index=False)

16179.920885674475 (array(38.45), array(350.95))
Latitude:  38.45
Longitude:  350.95
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
3892.164927419531 (array(40.45), array(351.45))
Latitude:  40.45
Longitude:  351.45
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
15455.6224090985 (array(40.7), array(352.7))
Latitude:  40.7
Longitude:  352.7
04 30
05 01
05 02


/tmp/ipykernel_22827/98418508.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_22827/98418508.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_22827/98418508.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
8227.017390676696 (array(41.7), array(351.7))
Latitude:  41.7
Longitude:  351.7
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
9688.278805368242 (array(41.95), array(352.95))
Latitude:  41.95
Longitude:  352.95
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08


/tmp/ipykernel_22827/98418508.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_22827/98418508.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_22827/98418508.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
13890.007071131957 (array(41.95), array(351.45))
Latitude:  41.95
Longitude:  351.45
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
7519.267877791736 (array(41.45), array(352.7))
Latitude:  41.45
Longitude:  352.7
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
5920.903491219621 (array(41.2), array(352.45))
Latitude:  41.2
Longitude:  352.45
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
11551.341910670033 (array(38.45), arr

...

/tmp/ipykernel_22827/98418508.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_22827/98418508.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_22827/98418508.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
8475.22598205075 (array(40.45), array(352.45))
Latitude:  40.45
Longitude:  352.45
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
11227.84270446845 (array(39.95), array(351.7))
Latitude:  39.95
Longitude:  351.7
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20


/tmp/ipykernel_22827/98418508.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_22827/98418508.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_22827/98418508.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31
13133.263133212198 (array(39.45), array(350.7))
Latitude:  39.45
Longitude:  350.7
04 30
05 01
05 02
05 03
05 04
05 05
05 06
05 07
05 08
05 09
05 10
05 11
05 12
05 13
05 14
05 15
05 16
05 17
05 18
05 19
05 20
05 21
05 22
05 23
05 24
05 25
05 26
05 27
05 28
05 29
05 30
05 31


/tmp/ipykernel_22827/98418508.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_22827/98418508.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_22827/98418508.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_